# Chapter 5: Advancing Multi-Agent Systems with Pydantic AI

In this chapter, we will explore the next level of AI agent development using Pydantic AI. Rather than just handling single queries, we'll learn how to orchestrate multiple agents, build intelligent workflows and add memory for long-term interactions

Before we start, we need to set up the environment.

In [1]:
import nest_asyncio
nest_asyncio.apply()

from pydantic_ai import Agent, RunContext
import os
import dotenv

dotenv.load_dotenv()

# Set your Google API key
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

## 1. Multi-Agent Collaboration

Imagine building an AI-powered startup team where different agents collaborate just like real employees:

- The CEO Agent delegates tasks.
- The Marketing Agent generates ad campaigns.
- The Finance Agent manages budgets.
- The Tech Lead Agent handles product development.

In [18]:
# CEO Agent - The boss who assigns tasks
ceo_agent = Agent(
    'google-gla:gemini-1.5-flash',
    # The system prompt is so important for this kind of multi-agent collaboration. You need to specify what are the tools available to the agents and what are the constraints.
    system_prompt="You are the CEO of a tech startup. Based on user input, decide whether the Marketing, Finance, or Tech Lead should handle the task. then call to the `assign_task` tool to delegate the task."
)

# Marketing Agent - Creates ad campaigns
marketing_agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt="You are a marketing expert. Generate creative ad campaigns."
)

# Finance Agent - Manages budgets
finance_agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt="You are a financial expert. Handle budgets and financial planning."
)

# Tech Lead Agent - Handles product development
tech_agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt="You are the tech lead. Guide product development and fix technical issues."
)

# CEO assigns tasks
@ceo_agent.tool
async def assign_task(ctx: RunContext, task: str, assignee: str = None) -> str:
    """Delegates tasks to the appropriate agent and returns their response in a detailed format.
    
    Args:
        task: The task to be completed
        assignee: Optional - The team member to assign (Marketing, Finance, or Tech)
    """
    # Determine which agent to use based on assignee or task content
    if assignee and "market" in assignee.lower() or "marketing" in task.lower():
        r = await marketing_agent.run(task)
        print("Marketing Agent's response:", r.data)
        return r.data   
    elif assignee and "finance" in assignee.lower() or "budget" in task.lower() or "finance" in task.lower():
        r = await finance_agent.run(task)
        print("Finance Agent's response:", r.data)
        return r.data
    elif assignee and "tech" in assignee.lower() or "tech" in task.lower() or "product" in task.lower():
        r = await tech_agent.run(task)
        print("Tech Lead Agent's response:", r.data)
        return r.data
    else:
        return "I don't recognize this task or assignee."

# Test the AI Startup Team
# response = await ceo_agent.run("Create a marketing campaign for our new AI app.")
response = await ceo_agent.run("Create a financial plan for our new AI app.")
print("CEO's response:", response.data)

Finance Agent's response: ## Financial Plan for a New AI App

This financial plan outlines the key financial aspects of launching and sustaining a new AI application.  It requires specific information about your app to be fully fleshed out, but provides a robust framework.  You'll need to replace the bracketed information with your own data.

**I. Executive Summary:**

[Insert a brief summary of your AI app, its target market, and your key financial goals (e.g., profitability within 18 months, achieving $X million in revenue within 3 years).]

**II. Market Analysis:**

* **Target Market:**  Define your target audience (demographics, psychographics, needs).  Include market size estimates and growth potential.  [Insert detailed market analysis, including competitor analysis.]
* **Market Segmentation:**  How will you segment your market to tailor your marketing efforts and pricing strategies? [Describe your segmentation strategy.]
* **Competitive Landscape:**  Identify key competitors and

> Remark: The `assign_task` tool is called to delegate the task to the appropriate agent.

System prompt is so important for this kind of multi-agent collaboration. You need to specify what are the tools available to the agents and what are the constraints.


## 2. Agent-Guided Verification with Handoff

Agent verifies user details, prompts for missing information, and enforces retry limits before either completing verification or handing off the process (e.g., terminating or escalating to a human).

In [4]:
from typing import Union
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from pydantic_ai.usage import Usage, UsageLimits


class UserDetails(BaseModel):
    name: str
    age: int = Field(ge=1, le=120)
    matric_number: str


class Failed(BaseModel):
    """User failed to provide sufficient details after multiple attempts."""


# Define an agent using Gemini for verification
user_verification_agent = Agent[None, Union[UserDetails, Failed]](
    "google-gla:gemini-1.5-flash",
    result_type=Union[UserDetails, Failed],  # type: ignore
    system_prompt=(
        "Extract the user's name, age, and matric number for verification. "
        "If any information is missing or incomplete, request clarification up to three times."
    ),
)

usage_limits = UsageLimits(request_limit=3)  # Limit AI attempts to 3


async def verify_user(usage: Usage) -> Union[UserDetails, None]:
    message_history: Union[list[ModelMessage], None] = None

    for i in range(3):
        answer = input("Please provide your name, age, and matric number for verification:")
        print("User's input attempt", i+1, ":", answer)

        result = await user_verification_agent.run(
            answer,
            message_history=message_history,
            usage=usage,
            usage_limits=usage_limits,
        )

        if isinstance(result.data, UserDetails):
            return result.data
        else:
            print("Incomplete details. Please try again.")
            message_history = result.all_messages(
                result_tool_return_content="Ensure you provide your full name, age, and matric number."
            )

    print("Verification failed after multiple attempts. Process terminated.")
    return None


async def main():
    usage: Usage = Usage()
    user_details = await verify_user(usage)

    if user_details is not None:
        print(f"User verified: {user_details.name}, Age: {user_details.age}, Matric No: {user_details.matric_number}")


# Run the main function if this file is executed
if __name__ == "__main__":
    import asyncio
    asyncio.run(main())


User's input attempt 1 : ssyok
Incomplete details. Please try again.
User's input attempt 2 : i am 21 year old
Incomplete details. Please try again.
User's input attempt 3 : my matric number is 23005023 and my name is the first input 
User verified: ssyok, Age: 21, Matric No: 23005023


## Conclusion

Through these examples, we saw how multi-agent orchestration enhances AI workflows. Whether it’s automating business operations or enforcing structured verification, Pydantic AI allows for efficient, intelligent, and scalable AI-driven systems.

This knowledge sets the foundation for building more complex agent ecosystems—from AI-powered teams to self-sufficient decision-making systems. 